# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233591726285                   -0.50    7.0
  2   -7.250021798074       -1.78       -1.41    1.0
  3   -7.251317967737       -2.89       -2.02    4.0
  4   -7.251314016743   +   -5.40       -2.42    4.0
  5   -7.251316480942       -5.61       -2.48    3.0
  6   -7.251337075248       -4.69       -3.04    2.0
  7   -7.251338772659       -5.77       -4.09    2.0
  8   -7.251338792180       -7.71       -4.14    4.0
  9   -7.251338797671       -8.26       -4.50    2.0
 10   -7.251338798106       -9.36       -4.65    1.0
 11   -7.251338798697       -9.23       -5.53    3.0
 12   -7.251338798701      -11.38       -5.65    2.0
 13   -7.251338798703      -11.86       -5.87    1.0
 14   -7.251338798704      -11.71       -6.73    3.0
 15   -7.251338798705      -13.27       -6.88    3.0
 16   -7.251338798705      -14.01       -7.17    1.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.035387809119208115
[ Info: Arnoldi iteration step 2: normres = 0.5283207302804925
[ Info: Arnoldi iteration step 3: normres = 0.6460456563688416
[ Info: Arnoldi iteration step 4: normres = 0.6362566657725816
[ Info: Arnoldi iteration step 5: normres = 0.35920196326317416
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.01e-01, 6.67e-02, 2.61e-01, 2.15e-01, 1.17e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4144042123404512
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.47e-02, 1.07e-01, 3.70e-01, 1.16e-01, 7.44e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06966006624154675
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.24e-03, 1.09e-02, 2.14e-02, 2.95e-02, 5.13e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12471570833146566
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.69e